In [9]:
# T: Just runs from -6 to 6, creates the curve shape when passed through the sigmoid function
# T2: Sets up the x-coordinates to run from position to parent position, gets iterated similar to T but specific to the end_node_level and start_node_level
# P1: This is the y-coordinate of the parent position
# P2: This is the y-coordinate of the position
# Branch: Still trying to figure out exactly why this field is so important to the Tableau viz...
# Stat/Dyn - stat shows the whole org chart, dyn only allows nodes to expand/contract

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS

# https://stackoverflow.com/questions/44549110/python-loop-through-excel-sheets-place-into-one-df

data_file_loc = 'C:/Users/jlahrman/OneDrive - LMI/Documents/DoS_Viz/Data/data_for_tableau/'
filename = '2020_05_14.xlsx'

sheets = pd.read_excel(data_file_loc + filename, sheet_name = None)

# blank data frame to append every sheet
final_data = pd.DataFrame()

# rename level to 'end_node_level'
for name, sheet in sheets.items():
    sheet['sheet'] = name
    sheet = sheet.rename(columns=lambda x: x.split('\n')[-1])
    if name != 'query_results':
        sheet.columns = ['row_id','track','position_category','position_name','composition', 'detail_count', 'pos_no','to_plot','inner_count','reports_to_row_id','end_node_level','department']
        final_data = final_data.append(sheet)

# start a loop here to handle each individual department, and a data frame to hold all final data
final_curve_data = pd.DataFrame()
department_list = np.sort(final_data.department.unique())
exception_report = pd.DataFrame()

for x in range(len(department_list)):
    print('department', x+1, 'of', len(department_list),':',department_list[x])
    department_data = final_data[final_data.department == department_list[x]]
   
    # add in the "unknown" level 0 node, restart each department with an end_node_id of 1000 so they're completely unique
    zero_row = {'row_id': 0,
                'track': 0,
                'position_category': 'Unknown',
                'position_name': 'Unknown',
                'composition': 'Unknown',
                'detail_count': 0,
                'pos_no': 0,
                'inner_count': 0,
                'reports_to_row_id': 0,
                'to_plot': 1,
                'department': department_list[x],
                'end_node_level': 0
               }
    # append_zero_row
    department_data = department_data.append(zero_row, ignore_index = True)
    # Joel: Add records that are plotted but do not have a track number to the exception report
    # These records don't actually have to be removed, it won't mess up the plots, but they should be called out
    no_level_records = department_data[department_data['end_node_level'].isnull() & department_data['to_plot'] == 1]
    no_level_records['exception_report_composition'] = 'plotted record with no level'
    exception_report = exception_report.append(no_level_records)
    # have any record with a blank 'reports_to_id' report to the unknown position (with row_id 0)
    department_data['reports_to_row_id'] = department_data['reports_to_row_id'].fillna(0)
    # sort by row_id
    department_data = department_data.sort_values(by = ['row_id'], ascending = True)
    
    # Joel: Resort the records to align the points.
    #1. Add two blank fields to the data frame: "sorter" and "reports_to_sorter"
    #2. Filter the data to be only the positions to plot
    #3. Create a unique sorted list of the tracks in the data, to be used to loop through top to bottom
    #4. First row of n positions doesn't need to be sorted, gets sorter value and reports_to_sorter values of 1 through n. Hopefully n will always be 1 (i.e., only one position will be at very top)
    #5. Save that first row as a new data frame.
    #6. Loop through each track value in the unique list (sorted ascending)
        #a. Create a subsetted data frame with track matching the list value
        #b. For each value, look up the sorter of the record of the parent in the new data frame, and enter that as the reports_to_sorter of the record
        #c. Sort the subsetted data by the reports_to_sorter ascending
        #d. Loop through the records, incrementing the sorter by 1 each time (keeping the incrementer from the previous subset)
        #e. Append the data to the original 0 data frame
    #6. Sort the whole data frame by sorter ascending
    #7. Replace the origial department_data data frame with this new sorted one.
    #8. Replace row_id with sorter, and reports_to_row_id with reports_to_sorter
    #9. Now append all the records that we don't want to plot. Look up their reports_to_row_id values in the sorter and replace.
    
    # iterate through the levels in reverse order - will probably be better to do it this way when trying to center the parent nodes

    # determine maximum level in the data
    max_level = int(department_data['end_node_level'].max())
    # create blank data frame to append the records with updated fields
    final_data_with_coords = pd.DataFrame()
    # create a counter for the end_node_id, this just needs to stay unique in the data frame. Restart each department at 1000 so they're unique
    end_node_id = (x+1)*1000+1

    # Set a value to iterate the P2 coordinate for each row
    P2_static_iterator = 3
    # Set the value of T to interate both T and T2
    T_initializer = -6
    T_iterator = 6 # I think this should be right, it's just 6? Probably only need one variable here
    
    # set up a list to eventually turn into a data frame that we'll use to center the points on each row
    level_max = []
    
    for i in range(0,max_level+1): # got to add 1 here, otherwise it won't get down to the level 0 "unknown"
        level_i = max_level - i
        # we also only need to attach these coordinates to values that we want to plot
        subset = department_data[(department_data.end_node_level == level_i) & (department_data.to_plot == 1)]
        # start P2 at 0, then append values, iterating by the P2_iterator value
        P2_static_coord = 0
        P2_static_set = []
        end_node_id_set = []
        # for each entry, increment P2 (the x-coordinate of the node) by 3
        # for each entry, increment end_node_id by 1
        for j in range(len(subset)):
            P2_static_set.append(P2_static_coord)
            end_node_id_set.append(end_node_id)
            P2_static_coord += P2_static_iterator
            end_node_id += 1
        if len(subset) > 0:
            # have to subtract that iterator back out because it already added the iterator value before the loop ended...
            level_max.append([department_list[x],level_i,P2_static_coord - P2_static_iterator]) 

    # Append the individual lists we just created as new columns of the data set using 'assign'
        subset = subset.assign(P2_static = P2_static_set)
        subset = subset.assign(end_node_id = end_node_id_set)
        # add the T_initializer value for the initial T
        subset = subset.assign(T = T_initializer)
        final_data_with_coords = final_data_with_coords.append(subset)
                
    # here is where I use that level_max data that we compiled above to adjust the P2s to be centered on the page 
    # we have to finalize the P2s here because everyone's P1 coordinate is based on the P2 of their parent position
    # determine the maximum P2 value for each level
    level_max = pd.DataFrame(level_max,columns=['department','level','max_P2_static'])
    # now find the overall maximum value for the department
    overall_max = level_max['max_P2_static'].max()
    # to horizontally center, determine a level-specific adjustment factor
    level_max['P2_static_adjustment'] = (overall_max - level_max['max_P2_static'])/2

    # maybe this could be combined with the loop below but I'll keep separate for now.
    # go through each record in final_data_with_coords, look up the appropriate P2_adjustment for the level, and add it to P2

    for j in range(len(final_data_with_coords)):
        # go through each row individually, save the row as a series
        lookup = final_data_with_coords.iloc[j]
        # find the P2_static value for that value
        end_node_level_lookup = lookup.get(key = 'end_node_level')
        # now find it in the level_max_data frame, and add that value to the existing value of P2
        final_data_with_coords.at[final_data_with_coords.index[j], 'P2_static'] += level_max[level_max['level']==end_node_level_lookup]['P2_static_adjustment']
    
    # now we need to get some information from the position to which each position reports to:
    # to get the start node level and id
    # reports to composition
    # And P2 of the parent'scomposition node becomes P1 of the position
    # we have to look up the end node level and id of the position listed in the 'reports_to_row_id' field

    final_data_with_coords['reports_to_composition'] = ''
    final_data_with_coords['P1_static'] = ''
    final_data_with_coords['start_node_id'] = ''
    final_data_with_coords['start_node_level'] = ''
    final_data_with_coords['T2'] = ''

    for j in range(len(final_data_with_coords)):
        # go through each row individually, save the row as a series
        try:
            lookup = final_data_with_coords.iloc[j]
            # now determine the reports_to_row_id value within the series
            lookup_row_id = lookup.get(key = 'reports_to_row_id')

            if pd.isna(lookup_row_id):
                # if this position doesn't have a known report up, just list the position's name as the one it reports to
                # Start node level and id are the same as the position's end node level and id - hold on there, it needs to be decremented back one but we'll handle that later
                # And we can't create T2 until here, the first value has to be based on its parent's position unless we want to increment backwards

                final_data_with_coords['reports_to_composition'].iloc[j] = lookup.get(key = 'composition')
                final_data_with_coords['P1_static'].iloc[j] = lookup.get(key = 'P2_static')
                final_data_with_coords['start_node_id'].iloc[j] = lookup.get(key = 'end_node_id')
                final_data_with_coords['start_node_level'].iloc[j] = lookup.get(key = 'end_node_level')

                # subtracting 1 from the end_node_level eventually allows T2 to start at -6
                # I'm not sure there is a good reason for this except to have it match T which runs from -6 to 6
                final_data_with_coords['T2'].iloc[j] = T_initializer + (lookup.get(key = 'end_node_level')-1) * 2 * T_iterator
            else:
                # if this position does have a report up, narrow the original data frame to the row_id with the lookup value
                reports_to_info = final_data_with_coords[final_data_with_coords['row_id'] == lookup_row_id]
                final_data_with_coords['reports_to_composition'].iloc[j] = reports_to_info['composition'].iloc[0]
                final_data_with_coords['P1_static'].iloc[j] = reports_to_info['P2_static'].iloc[0]
                final_data_with_coords['start_node_id'].iloc[j] = reports_to_info['end_node_id'].iloc[0]
                final_data_with_coords['start_node_level'].iloc[j] = reports_to_info['end_node_level'].iloc[0]
                final_data_with_coords['T2'].iloc[j] = T_initializer + (reports_to_info['end_node_level'].iloc[0]-1) * 2 * T_iterator        
        except:
            final_data_with_coords['reports_to_composition'].iloc[j] = 'exception'
                            
    # Send "exception" to exception report
    reports_to_unplotted_node = final_data_with_coords[final_data_with_coords.reports_to_composition == 'exception']
    reports_to_unplotted_node = reports_to_unplotted_node[['department','composition','end_node_id','end_node_level','pos_no','position_category','position_name','reports_to_composition','reports_to_row_id','row_id','start_node_id','start_node_level']]
    reports_to_unplotted_node['exception_report_composition'] = 'reports to unplotted parent'
    exception_report = exception_report.append(reports_to_unplotted_node)
    # Then remove "remove" from final_data_with_coords
    final_data_with_coords = final_data_with_coords[final_data_with_coords.reports_to_composition != 'exception']
                        
    # These fields were initiated as text, need to convert to numeric
    final_data_with_coords[['P1_static', 'start_node_id', 'start_node_level','T2']] = final_data_with_coords[['P1_static', 'start_node_id', 'start_node_level','T2']].apply(pd.to_numeric)
        
    # Sort by row_id ascending.
    final_data_with_coords = final_data_with_coords.sort_values(by = ['row_id'], ascending = True)
    # If no parent, or parent marked with "to remove", then mark as "remove".
    # Have to ignore row_id 0 though, otherwise it will cascade down to everything!
    for w in range(len(final_data_with_coords)):
        lookup = final_data_with_coords['start_node_id'].iloc[w]
        end_node_id_lookup = final_data_with_coords[final_data_with_coords.end_node_id == lookup]
        try:
            if end_node_id_lookup['row_id'].iloc[0] != 0 and end_node_id_lookup['reports_to_composition'].iloc[0] == 'remove':
                final_data_with_coords['reports_to_composition'].iloc[w] = 'remove'
        except:
            final_data_with_coords['reports_to_composition'].iloc[w] = 'remove'
    # Send "remove" to exception report
    ascending_wash_upstream_unplotted_parent = final_data_with_coords[final_data_with_coords.reports_to_composition == 'remove']
    ascending_wash_upstream_unplotted_parent = ascending_wash_upstream_unplotted_parent[['department','composition','end_node_id','end_node_level','pos_no','position_category','position_name','reports_to_composition','reports_to_row_id','row_id','start_node_id','start_node_level']]
    ascending_wash_upstream_unplotted_parent['exception_report_composition'] = 'unplotted upstream parent - ascending wash'
    exception_report = exception_report.append(ascending_wash_upstream_unplotted_parent)
    # Then remove "remove" from final_data_with_coords
    final_data_with_coords = final_data_with_coords[final_data_with_coords.reports_to_composition != 'remove']
            
    # Sort by row_id descending and repeat
    # Joel - if you were even remotely competent you would build a function and pass the sorted data frame in twice
    final_data_with_coords = final_data_with_coords.sort_values(by = ['row_id'], ascending = False)
    # If no parent, or parent marked with "to remove", then mark as "to remove"
    for w in range(len(final_data_with_coords)):
        lookup = final_data_with_coords['start_node_id'].iloc[w]
        end_node_id_lookup = final_data_with_coords[final_data_with_coords.end_node_id == lookup]
        try:
            if end_node_id_lookup['row_id'].iloc[0] != 0 and end_node_id_lookup['reports_to_composition'].iloc[0] == 'remove':
                final_data_with_coords['reports_to_composition'].iloc[w] = 'remove'
        except:
            final_data_with_coords['reports_to_composition'].iloc[w] = 'remove'
    # Send "remove" to exception report
    descending_wash_upstream_unplotted_parent = final_data_with_coords[final_data_with_coords.reports_to_composition == 'remove']
    descending_wash_upstream_unplotted_parent = descending_wash_upstream_unplotted_parent[['department','composition','end_node_id','end_node_level','pos_no','position_category','position_name','reports_to_composition','reports_to_row_id','row_id','start_node_id','start_node_level']]
    descending_wash_upstream_unplotted_parent['exception_report_composition'] = 'unplotted upstream parent - descending wash'
    exception_report = exception_report.append(descending_wash_upstream_unplotted_parent)
    # Then remove "remove" from final_data_with_coords
    final_data_with_coords = final_data_with_coords[final_data_with_coords.reports_to_composition != 'remove']    

    # Build a loop that creates P1_dynamic and P2_dynamic, as well as compiles the Path.
    # Also add an empty column of lists to which end_node_ids can be appended to build the path variable
    # Add a parent_path in case we want to toggle between paths on the viz
    
    final_data_with_coords['path'] = ''
    final_data_with_coords['parent_path'] = ''
    
    # start by setting P1_ and P2_dynamic to equal their static counterparts for level 1 positions and add the end_node_id to the path
    # Zctually, let's do that for any node whose end_node_id = start_node id (ie, does not report to anyone)
    # BUT, because we now have the "unknown" zero level, the block below only really gets applied to level 0
    # since that is the only node for which the start node and end node have the same level
    # if we wanted to "lock" additional levels at the top of the viz we could do it in this section, probably wouldn't be too hard to adjust
    for y in range(len(final_data_with_coords)):
        if final_data_with_coords.at[final_data_with_coords.index[y], 'end_node_level'] == final_data_with_coords.at[final_data_with_coords.index[y], 'start_node_level']:
            final_data_with_coords.at[final_data_with_coords.index[y], 'P1_dynamic'] = final_data_with_coords.at[final_data_with_coords.index[y], 'P1_static']
            final_data_with_coords.at[final_data_with_coords.index[y], 'P2_dynamic'] = final_data_with_coords.at[final_data_with_coords.index[y], 'P2_static']

    # for levels not at the head, find the number of records that have that level and have a report (IE, they do NOT report to themselves)
    # HAVE to sort by start_node_id descending, that way it starts with the higher end_node_id values which are the higher levels
    P2_dataset = final_data_with_coords[(final_data_with_coords['end_node_level'] != final_data_with_coords['end_node_level'].min()) & (final_data_with_coords['end_node_id'] != final_data_with_coords['start_node_id'])]
    P2_start_node_id = P2_dataset['start_node_id'].value_counts().rename_axis('start_node_id').reset_index(name = 'total')
    P2_start_node_id['counter'] = 0
    P2_start_node_id = P2_start_node_id.sort_values(by = ['start_node_id'], ascending = False)
        
    # Now with P2_start_node_id, we have to loop through the original data set to find all records with the start_node_id
    for g in range(len(P2_start_node_id)):
        for h in range(len(final_data_with_coords)):
            if ((final_data_with_coords['start_node_id'].iloc[h] == P2_start_node_id['start_node_id'].iloc[g]) and (final_data_with_coords['start_node_id'].iloc[h] != final_data_with_coords['end_node_id'].iloc[h])):
                # set P1 to match P2 of parent. Look up start_row_id of the position
                lookup = final_data_with_coords.iloc[h]
                end_node_id_lookup = lookup.get(key = 'start_node_id')
                # now find the matching end_node_id for the parent, and set that parent's P2 to be the child's P1
                parent_info = final_data_with_coords[final_data_with_coords.end_node_id == end_node_id_lookup]
                try:
                    final_data_with_coords['P1_dynamic'].iloc[h] = parent_info['P2_dynamic'].iloc[0]
                    # also set it to be the initial P2 for the child, but have to adjust so that the row are spread horizontally
                    final_data_with_coords['P2_dynamic'].iloc[h] = parent_info['P2_dynamic'].iloc[0] + P2_start_node_id['counter'].iloc[g] - (P2_start_node_id['total'].iloc[g]-1)/2
                    P2_start_node_id['counter'].iloc[g] += 1
                    # bring in the path from the parent
                    final_data_with_coords['path'].iloc[h] = parent_info['path'].iloc[0]
                    final_data_with_coords['parent_path'].iloc[h] = parent_info['path'].iloc[0]
                    # append the end_node_id of the current row
                    # in case of skipping levels, add in '-0' so that positions on the same level always have the same number of characters on their path
                    filler_length = (final_data_with_coords['end_node_level'].iloc[h] - final_data_with_coords['start_node_level'].iloc[h]-1).astype(int)
                    filler_string = '-0'
                    if filler_length > 0:
                        filler = filler_string * filler_length
                    else:
                        filler = ''
                    final_data_with_coords['path'].iloc[h] = final_data_with_coords['path'].iloc[h] + filler_string * filler_length + '-' + str(int(final_data_with_coords['end_node_id'].iloc[h]))
                except:
                    exception_report_item = final_data_with_coords.iloc[h]
                    exception_report_item = exception_report_item[['department','composition','end_node_id','end_node_level','pos_no','position_category','position_name','reports_to_composition','reports_to_row_id','row_id','start_node_id','start_node_level']]
                    exception_report['exception_report_composition'] = 'unplotted upstream parent - how did this slip through'
                    exception_report = exception_report.append(exception_report_item)
                    final_data_with_coords['reports_to_composition'].iloc[h] = 'exception'
                    
   # Remove data from final_data_with_coords thave have an upstream unplotted parent
    final_data_with_coords = final_data_with_coords[(final_data_with_coords.reports_to_composition != 'exception')]

    # Add exceptions for positions that report laterally
    lateral_reports = final_data_with_coords[(final_data_with_coords.end_node_level == final_data_with_coords.start_node_level) & (final_data_with_coords.end_node_id != final_data_with_coords.start_node_id)]
    if len(lateral_reports) > 0:
        lateral_reports = lateral_reports[['department','composition','end_node_id','end_node_level','pos_no','position_category','position_name','reports_to_composition','reports_to_row_id','row_id','start_node_id','start_node_level']]
        lateral_reports['exception_report_composition'] = 'reports laterally'
        exception_report = exception_report.append(lateral_reports)
    # Add exceptions for positions that report laterally
    downward_reports = final_data_with_coords[final_data_with_coords.end_node_level < final_data_with_coords.start_node_level]
    if len(downward_reports) > 0:
        downward_reports = downward_reports[['department','composition','end_node_id','end_node_level','pos_no','position_category','position_name','reports_to_composition','reports_to_row_id','row_id','start_node_id','start_node_level']]
        downward_reports['exception_report_composition'] = 'reports downward'
        exception_report = exception_report.append(downward_reports)
        
    # Need to add final hyphens, allow for 50 hyphens which requires 51 total hyphens
    allowed_levels = 50
    for j in range(len(final_data_with_coords)):
        needed_hyphens = int(allowed_levels + 1 - final_data_with_coords['end_node_level'].iloc[j])
        final_data_with_coords['path'].iloc[j] = str(final_data_with_coords['path'].iloc[j]) + '-' * needed_hyphens
        needed_parent_hyphens = int(allowed_levels + 1 - final_data_with_coords['start_node_level'].iloc[j])
        final_data_with_coords['parent_path'].iloc[j] = str(final_data_with_coords['parent_path'].iloc[j]) + '-' * needed_parent_hyphens
 
    # Now we've got all the coordinates set, we can build the 49 records needed to create the sigmoid effect by incrementing T2
    # If a position does not report to another position, then T2 needs to be incremented down so that the Y value of the start node gets incremented down to the previous level
    # final_data_with_coords['T2'].where(~(final_data_with_coords.end_node_level == final_data_with_coords.start_node_level), other=final_data_with_coords.T2 + 2 * T_initializer, inplace=True)

    branch_set = []

    for j in range(len(final_data_with_coords)):
        lookup = final_data_with_coords.iloc[j]
        branch_item = "Level " + str(int(lookup.get(key = 'start_node_level'))) + " (" + str(int(lookup.get(key = 'start_node_id'))) + ") - Level " + str(int(lookup.get(key = 'end_node_level'))) + " (" + str(int(lookup.get(key = 'end_node_id'))) + ")"
        branch_set.append(branch_item)
    final_data_with_coords = final_data_with_coords.assign(branch = branch_set)

    # add in a T2 iterator
    final_data_with_coords['T2_iterator'] = .25 * (final_data_with_coords['end_node_level'] - final_data_with_coords['start_node_level'])

    # annoying but I'm going to create a unique list of IDs and then filter the data frame by every unique value
    # should have the same effect as selecting individual rows (which is a pain because those rows are saved as series and I lose the column names)
    
    # first we can remove "head" nodes - positions to which other positions report but do not report to any positions themselves
    # actually, let's not remove them, which would throw off the headcounts. But we won't iterate them.
    # they will have nodes drawn by the positions that report to them
    # these will be identified by two characteristics:
    # 1) start_node_id = end_node_id, indicating that the position does not report to another position
    # 2) the count of start_node_ids in the dataset matching the position's dataset is greater than one, indicating that the position has subordinates

    # first determine the number of reports each position has
    number_reporting_to_start_node_id_set = []    
    
    # strip leading and trailing whitespace from selected fields
    final_data_with_coords['position_category'] = final_data_with_coords['position_category'].str.strip()
    
    # determine number of reports for each position, append to number_reporting_to_start_node_id_set
    for i in range(len(final_data_with_coords)):
        #Should pos_snid be end_node_id, not start_node_id?
        #Otherwise we're finding how many people report to the person this position reports to.
        #OK ONLY if we're using to find whether it's a head node.
        pos_snid = final_data_with_coords['start_node_id'].iloc[i]
        reports_count = final_data_with_coords.apply(lambda x: True if x['start_node_id']==pos_snid else False, axis = 1)
        num_reports = len(reports_count[reports_count==True].index)
        number_reporting_to_start_node_id_set.append(num_reports)
    # append the number_reporting_to_start_node_id_set to final_data_with coords.
    final_data_with_coords = final_data_with_coords.assign(number_reporting_to_start_node_id = number_reporting_to_start_node_id_set)
    # move the "unknown" nodes with an end_node_level of 0 to the upper left corner
    # actually maybe keep them in the middle, might make it easier to plot things later
#    final_data_with_coords.loc[final_data_with_coords['end_node_level'] == 0, ['P2_static','P2_dynamic']] = 0    
#    final_data_with_coords.loc[final_data_with_coords['start_node_level'] == 0, ['P1_static','P1_dynamic']] = 0    
    # create a boolean to find records where number of reports > 1 and start_node_id = end_node_id
    final_data_with_coords['head_position'] = (final_data_with_coords['start_node_id']==final_data_with_coords['end_node_id']) & (final_data_with_coords['number_reporting_to_start_node_id']>1)
    # records to remove are TRUE, only keep records where that boolean is FALSE
    head_nodes = final_data_with_coords[final_data_with_coords['head_position']==True]
    head_nodes['node'] = 1
    head_nodes['node_type'] = 'head'

    # Filter head nodes out, remove all coordinates, and append them back in later
    # At this point only the "Unknown" level 0 nodes should be head nodes
    final_data_with_coords = final_data_with_coords[final_data_with_coords['head_position']==False]

    # determine number of PLOTTED reports for each position, append to data frame
    # this is to make it easy for Tableau to differentiate between managers and non-managers for visual purposes
    # determine number of nodes in the level for each position, append to data frame
    # this will probably come in handy in Tableau
    final_data_with_coords['plotted_reports'] = ''
    final_data_with_coords['nodes_in_level'] = ''    
    for i in range(len(final_data_with_coords)):
        reportees_count = final_data_with_coords.apply(lambda x: True if x['start_node_id']==final_data_with_coords['end_node_id'].iloc[i] else False, axis = 1)
        nodes_in_level_count = final_data_with_coords.apply(lambda x: True if x['end_node_level']==final_data_with_coords['end_node_level'].iloc[i] else False, axis = 1)
        final_data_with_coords['plotted_reports'].iloc[i] = len(reportees_count[reportees_count==True].index)
        final_data_with_coords['nodes_in_level'].iloc[i] = len(nodes_in_level_count[nodes_in_level_count==True].index)

    final_data_with_coords.to_csv('final_data_with_coords' + department_list[x] + '.csv',index=False)
    
    # now the only rows left where the start_node_id matches the end_node_id should be "floating" points
    # filter those out, they won't have a curve so then don't need to be iterated below
    floating_nodes = final_data_with_coords[(final_data_with_coords.end_node_id == final_data_with_coords.start_node_id)]
    floating_nodes['node'] = 1
    floating_nodes['node_type'] = 'floating'
    line_data = final_data_with_coords[(final_data_with_coords.end_node_id != final_data_with_coords.start_node_id)]
    line_data['node'] = 1    
    line_data['node_type'] = 'connector'
    
    # but now let's relabel end nodes and overwrite that 'connector' value, will probably make life easier
    identify_end_nodes_set = []
    for i in range(len(line_data)):
        reports_count = line_data.apply(lambda x: True if x['start_node_id']==line_data['end_node_id'].iloc[i] else False, axis = 1)
        num_reports = len(reports_count[reports_count==True].index)
        identify_end_nodes_set.append(num_reports)
    # append the number_reporting_to_start_node_id_set to final_data_with coords. We'll remove it later after filtering out the rows
    line_data = line_data.assign(identify_end_nodes = identify_end_nodes_set)
    # create a boolean to find records where number of reports 0
    line_data['end_position'] = (line_data['identify_end_nodes']==0)   
    # change node_type for those records, delete unnecessary fields
    line_data.loc[line_data['end_position'] == True, 'node_type'] = 'end'
    line_data.drop(columns="identify_end_nodes", inplace = True)
    line_data.drop(columns="end_position", inplace = True)
    
# now create the iterated points for the sigmoid lines    

    department_curve_data = pd.DataFrame()
    number_of_rows = 49
    
    # create 49 of each record:
    for ind in line_data.index:
        curve_data = line_data.loc[line_data.index.repeat(number_of_rows)]
        curve_data['node'] = 0
        curve_data['node_type'] = 'na'
        
    # now go through each individual set of 49 records and iterate T1 (by 0.25) and T2 (by 0.25 * the number of levels)

    row_id_list = final_data_with_coords['row_id'].tolist()
    for k in range(len(row_id_list)):
        iterated_data = curve_data[curve_data['row_id'] == row_id_list[k]]
        T_new_set = []
        T2_new_set = []
        for l in range(len(iterated_data)):
            lookup = iterated_data.iloc[l]
            T_new_item = lookup.get(key = 'T') + l * .25
            T2_new_item = lookup.get(key = 'T2') + l * lookup.get(key = 'T2_iterator')
            T_new_set.append(T_new_item)
            T2_new_set.append(T2_new_item)
        iterated_data = iterated_data.assign(T = T_new_set)
        iterated_data = iterated_data.assign(T2 = T2_new_set)
        department_curve_data = department_curve_data.append(iterated_data)
    
    # now append back the records that are not being appended, so that we can still include them in headounts
    unplotted_data = department_data[department_data.to_plot != 1]
    department_curve_data = department_curve_data.append(unplotted_data)
    # then put in the floating nodes and head nodes: DON'T APPEND IF WE'RE DOING THE Unknown at the top
    department_curve_data = department_curve_data.append(floating_nodes)
#    department_curve_data = department_curve_data.append(head_nodes)
    # put the line_data back in, to reattach the standard notes
    # only need this if we're going to make additional "node" points for the end and connector nodes
#    department_curve_data = department_curve_data.append(line_data)
    # and append to the master set
    final_curve_data = final_curve_data.append(department_curve_data)

# remove unnecessary fields
final_curve_data.drop(columns='T2_iterator', inplace = True) # don't need to show
final_curve_data.drop(columns='number_reporting_to_start_node_id', inplace = True) # this was just to filter some records out, we don't need it in the final data
final_curve_data.drop(columns='head_position', inplace = True) # this was just to filter some records out, we don't need it in the final data
final_curve_data.drop(columns=['node','node_type'], inplace = True) # this was just to filter some records out, we don't need it in the final data
exception_report.drop(columns=['start_node_id','start_node_level','reports_to_composition'], inplace = True) # this was just to filter some records out, we don't need it in the final data
# convert a few columns to integer
final_curve_data[['row_id', 'end_node_level','detail_count','pos_no','inner_count','reports_to_row_id','start_node_id','start_node_level']] = final_curve_data[["row_id", "end_node_level","detail_count","pos_no","inner_count","reports_to_row_id","start_node_id","start_node_level"]].apply(pd.to_numeric)

# publish final curve data
#final_curve_data.to_csv('final_curve_data.csv',index=False)

# send a copy over to the dashboard file
file_loc = 'C:/Users/jlahrman/OneDrive - LMI/Documents/DoS_Viz/4_dashboard/'
final_curve_data.to_csv(file_loc + 'final_curve_data.csv',index=False)
exception_report.to_csv(file_loc + 'exception_report.csv',index=False)

print('exception report:',len(exception_report), 'rows')
# send a copy over to the dashboard file
#file_loc = 'C:/Users/jlahrman/OneDrive - LMI/Documents/DoS_Viz/4_dashboard_recast/'
#final_curve_data.to_csv(file_loc + 'final_curve_data_recast.csv',index=False)

# if we process the data more to remove zeroes we can copy it here...
# attempt to get rid of trailing zeroes (.0)
# final_curve_data_fixed = final_curve_data.applymap(lambda cell: int(cell) if str(cell).endswith('.0') else cell)
# final_curve_data_fixed.to_csv('final_curve_data_fixed.csv',index=False)

<class 'numpy.ndarray'>
department 1 of 10 : DHS


C:\Users\jlahrman\.conda\envs\py36\lib\site-packages\ipykernel_launcher.py:372: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\jlahrman\.conda\envs\py36\lib\site-packages\ipykernel_launcher.py:373: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


department 2 of 10 : DoC
department 3 of 10 : DoC orig
department 4 of 10 : DoE
department 5 of 10 : DoE orig
department 6 of 10 : DoI
department 7 of 10 : DoJ
department 8 of 10 : ED
department 9 of 10 : USDA
department 10 of 10 : zTest 1 simple
exception report: 0 rows


In [2]:
# come up with a way to set the P2 increment to make the plot a roughly consistently rectangular shape? Maybe not necessary

# space when reporting to different positions
# additional items to exception report:
    # start_node_level higher than end_node level (position reports to a lower level)
    # end_node_level reports to same end_node_level, different end_node_id (position reports horizontally)

# Do we need one point for head nodes, would that make aspects of the Tableau viz easier?